In [ ]:
# IMDB리뷰 분류
# 자연어처리(NLP) : 컴퓨터를 사용해 인간의 언어를 처리하는 분야
# 말뭉치(corpus) : 자연어 처리 분야에서의 훈련 데이터
# 토큰(token)
# 어휘사전

In [1]:
from tensorflow import keras

In [2]:
(train_input, train_target), (test_input, test_target) = keras.datasets.imdb.load_data(num_words=100)

In [3]:
train_input.shape

(25000,)

In [4]:
len(train_input[1]) # len(train_input[n])개의 단어로 이루어진 문장이 25000개

189

In [5]:
train_target[:10] # 1 : 긍정, 0 : 부정

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [6]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target)

In [7]:
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input, maxlen=100) # 학습을 시킬 때 일괄적으로 규격이 같아야 하므로 데이터의 길이를 맞춰줌
# `maxlen=` : 길이를 지정하면 길이가 긴 경우는 잘라내고(앞부분을 자름) 짧은 경우는 0으로 패딩(앞부분에 추가)
# 뒷부분을 잘라내고 싶으면 `truncating='post'`로 옵션 지정
# 뒷부분에 추가하고 싶으면 `padding='post'`로 옵션 지정

In [8]:
train_seq.shape

(18750, 100)

In [9]:
train_seq[0]

array([76,  2,  2,  2,  2,  2, 76,  2, 15,  2, 50, 26, 99,  2,  2,  7,  2,
        2, 11,  2,  2,  2, 17, 73, 60,  2, 12,  2, 30,  2, 13,  2, 40,  8,
       30,  2,  8,  2, 51, 84, 26,  2,  4,  2,  9,  2,  2,  2,  2,  2,  9,
        2, 50, 18,  4,  2, 10, 10,  2,  2,  9,  6,  2,  2, 22, 15, 13,  2,
       16,  6,  2,  2, 39,  2,  8,  2, 13, 62,  2,  2,  4,  2,  2,  2, 37,
        2,  2,  5,  2,  2, 56,  2,  2,  2,  2,  6,  2,  2,  7, 58],
      dtype=int32)

In [10]:
train_input[0][-10:]

[56, 2, 2, 2, 2, 6, 2, 2, 7, 58]

In [11]:
val_seq = pad_sequences(val_input, maxlen=100)

In [12]:
val_seq.shape

(6250, 100)

In [14]:
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(8, input_shape=(100, 300)))
model.add(keras.layers.Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# to_categorical() 원-핫 인코딩 : 정숫값을 배열에서 해당 정수 위치의 원소만 1이고 나머지는 모두 0으로 변환
train_oh = keras.utils.to_categorical(train_seq)

In [16]:
train_oh.shape

(18750, 100, 100)

In [17]:
train_oh[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
val_oh = keras.utils.to_categorical(val_seq)

In [19]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 8)                   │           2,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,481 (9.69 KB)

 Trainable params: 2,481 (9.69 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
# model.compile(
#     optimizer=rmsprop,
#     loss='binary_crossentropy', # 이진분류
#     metrics=['accuracy']
# )
# es = keras.callbacks.EarlyStopping(patience=3)
# result = model.fit(train_oh, train_target, epochs=50, validation_data=(val_oh, val_target), callbacks=[es])

# 단어 임베딩
- 순환 신경망에서 텍스트를 처리할 때 즐겨 사용
- 정수 데이터를 입력받음
- embedding() : 첫번째 매개변수는 어휘 사전의 크기, 두번째 매개변수는 임베딩 벡터의 크기, 세번째 매개변수는 입력 시퀀스의 길이 지정

# simpleRNN

In [23]:
model2 = keras.Sequential()

model2.add(keras.layers.Embedding(300, 16, input_length=100)) # 100개짜리 데이터가 300개, 300개의 데이터를 16개로 만들어줘
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

In [24]:
model2.compile(loss='binary_crossentropy', metrics=['accuracy'])
es = keras.callbacks.EarlyStopping(patience=2)
result = model2.fit(train_seq, train_target, epochs=100, validation_data=(val_seq, val_target), callbacks=[es])

Epoch 1/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.5408 - loss: 0.6867 - val_accuracy: 0.5602 - val_loss: 0.6837
Epoch 2/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.6221 - loss: 0.6531 - val_accuracy: 0.6581 - val_loss: 0.6325
Epoch 3/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.6340 - loss: 0.6432 - val_accuracy: 0.6595 - val_loss: 0.6302
Epoch 4/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.6490 - loss: 0.6362 - val_accuracy: 0.6640 - val_loss: 0.6267
Epoch 5/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.6594 - loss: 0.6269 - val_accuracy: 0.6528 - val_loss: 0.6314
Epoch 6/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.6561 - loss: 0.6306 - val_accuracy: 0.6638 - val_loss: 0.6259
Epoch 7/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.6566 - loss: 0.6312 - val_accuracy: 0.6442 - val_loss: 0.6429
Epoch 8/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.6682 - loss: 0